In [2]:
import warnings
import pandas as pd
import duckdb
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('~/Desktop/2023.csv')

In [4]:
df

,OR Case Number,Admit Dt/Tm,Discharge Dt/Tm,Case Start Dt/Tm,Patient In Room Dt/Tm,Surgery Start Dt/Tm,Surgery Stop Dt/Tm,Pt Out of Room Dt/Tm,Operating Room (Surg)
0,MSURG-2023-628,3/27/23 17:50,3/31/23 16:08,3/28/23 11:26,3/28/23 10:53,3/28/23 11:26,3/28/23 11:50,03/28/2023 12:01,02 MSURG OR
1,SOUTH-2023-1551,1/19/23 9:05,1/27/23 15:30,1/19/23 16:42,1/19/23 16:03,1/19/23 16:42,1/19/23 17:47,01/19/2023 17:56,10 South OR
2,SOUTH-2023-18785,8/9/23 10:54,8/11/23 23:35,8/10/23 15:56,8/10/23 14:52,8/10/23 15:56,8/10/23 20:34,08/10/2023 21:24,18 South OR
3,SOUTH-2023-22904,9/22/23 15:21,NaN,9/25/23 14:05,9/25/23 13:38,9/25/23 14:05,9/25/23 17:34,09/25/2023 17:50,02 South OR
4,SOUTH-2023-7409,3/30/23 15:28,3/30/23 15:29,3/30/23 16:55,3/30/23 16:39,3/30/23 16:55,3/30/23 20:52,03/30/2023 21:03,12 South OR
...,...,...,...,...,...,...,...,...,...
8451,SOUTH-2023-5475,3/4/23 15:52,3/30/23 15:17,3/8/23 14:49,3/8/23 14:24,3/8/23 14:49,3/8/23 16:03,03/08/2023 16:09,08 South OR
8452,SOUTH-2023-11818,5/21/23 13:15,5/21/23 19:32,5/21/23 15:07,5/21/23 14:37,5/21/23 15:07,5/21/23 17:28,05/21/2023 17:34,04 South OR
8453,SOUTH-2023-15479,6/19/23 7:55,7/7/23 8:35,7/3/23 11:43,7/3/23 11:05,7/3/23 11:43,7/3/23 13:18,07/03/2023 13:38,05 South OR
8454,SOUTH-2023-15479,6/19/23 7:55,7/7/23 8:35,7/3/23 11:43,7/3/23 11:05,7/3/23 11:43,7/3/23 13:18,07/03/2023 13:38,05 South OR


In [5]:
df['admit_dt'] = pd.to_datetime(df['Admit Dt/Tm'])
df['discharge_dt'] = pd.to_datetime(df['Discharge Dt/Tm'])
df['Admit Dt/Tm'] = pd.to_datetime(df["Admit Dt/Tm"])
df['Discharge Dt/Tm'] = pd.to_datetime(df["Discharge Dt/Tm"])
df['Case Start Dt/Tm'] = pd.to_datetime(df["Case Start Dt/Tm"])
df['Patient In Room Dt/Tm'] = pd.to_datetime(df["Patient In Room Dt/Tm"])
df['Surgery Start Dt/Tm'] = pd.to_datetime(df["Surgery Start Dt/Tm"])
df['Surgery Stop Dt/Tm'] = pd.to_datetime(df["Surgery Stop Dt/Tm"])
df['Pt Out of Room Dt/Tm'] = pd.to_datetime(df["Pt Out of Room Dt/Tm"])
df['operating_room'] = df['Operating Room (Surg)']

In [16]:
df['total_elapsed'] = (df['Pt Out of Room Dt/Tm'] - df['Patient In Room Dt/Tm'])/pd.Timedelta(minutes=1)
df['surgery_start_wait'] = (df['Surgery Start Dt/Tm'] - df['Patient In Room Dt/Tm'])/pd.Timedelta(minutes=1)
df['surgery_end_wait'] = (df['Pt Out of Room Dt/Tm'] - df['Surgery Stop Dt/Tm'])/pd.Timedelta(minutes=1)
df['surgery_active_time'] = (df['Surgery Stop Dt/Tm'] - df['Surgery Start Dt/Tm'])/pd.Timedelta(minutes=1)
df['wait'] = df['surgery_start_wait'] + df['surgery_end_wait']
df['percent_of_wait'] = df['wait']/df['total_elapsed'].fillna(0)

In [17]:
duckdb.sql("""SELECT * FROM df WHERE wait < 0""").df()    ## Data qauality issue. This one should be recoded or discarded. 

,OR Case Number,Admit Dt/Tm,Discharge Dt/Tm,Case Start Dt/Tm,Patient In Room Dt/Tm,Surgery Start Dt/Tm,Surgery Stop Dt/Tm,Pt Out of Room Dt/Tm,Operating Room (Surg),admit_dt,discharge_dt,operating_room,total_elapsed,surgery_start_wait,surgery_end_wait,surgery_active_time,wait,percent_of_wait
0,BOR-2023-1122,2023-02-08 06:52:00,2023-02-10 09:45:00,2023-02-08 13:58:00,2023-02-08 13:30:00,2023-02-08 13:58:00,2023-02-08 15:46:00,2023-02-08 13:59:00,OR 04 BRAN,2023-02-08 06:52:00,2023-02-10 09:45:00,OR 04 BRAN,29.0,28.0,-107.0,108.0,-79.0,-2.724138


In [7]:
duckdb.sql("""
select 
    operating_room, 
    avg(total_elapsed) as avg_total_elapsed, 
    avg(surgery_start_wait) as avg_surgery_start_wait,  
    avg(surgery_end_wait) as avg_surgery_end_wait,
    avg(surgery_active_time) as avg_surgery_active_time,
    avg(wait) as avg_wait, 
    avg(percent_of_wait) as avg_percent_of_wait,
    count(*)
from df 
group by 1 
having count(*) > 30
order by 6 desc""").df()

,operating_room,avg_total_elapsed,avg_surgery_start_wait,avg_surgery_end_wait,avg_surgery_active_time,avg_wait,avg_percent_of_wait,count_star()
0,11 South OR,176.262032,39.029412,14.064171,123.168449,53.093583,0.353090,374
1,03 MOC OR,130.036609,39.707129,12.448940,77.880539,52.156069,0.446875,519
2,14 South OR,189.040000,36.785366,14.145366,138.109268,50.930732,0.336847,1025
3,15 South OR,182.267925,35.731132,13.732075,132.804717,49.463208,0.340401,1060
4,OR 04 BRAN,121.169972,34.478754,11.509915,75.181303,45.988669,0.418227,353
5,04 MOC OR,116.481240,34.261011,10.991843,71.228385,45.252855,0.427335,613
6,18 South OR,123.644737,31.539474,13.500000,78.605263,45.039474,0.410982,76
7,09 South OR,146.322581,31.935484,13.000000,101.387097,44.935484,0.387351,31
8,OR 02 BRAN,119.389351,32.767055,11.259567,75.362729,44.026622,0.411898,601
9,01 MOC OR,105.473451,32.830383,9.831858,62.811209,42.662242,0.429032,678


In [8]:
df_dates = pd.DataFrame()
df_dates['datetime'] = pd.date_range("2022-10-14 0:00", "2023-10-14 23:59", freq="60min")

In [19]:
duckdb.sql("""
select 
    operating_room, 
    stddev(total_elapsed) as stddev_total_elapsed,
    stddev(wait) as stddev_wait, 
    stddev(percent_of_wait) as stddev_percent_of_wait,
    count(*)
from df 
group by 1 
having count(*) > 30
order by 3 desc""").df()

,operating_room,stddev_total_elapsed,stddev_wait,stddev_percent_of_wait,count_star()
0,18 South OR,82.994410,20.900042,0.126411,76
1,09 South OR,111.326363,19.106605,0.154580,31
2,11 South OR,86.628484,17.286245,0.147200,374
3,03 MOC OR,57.250589,16.734721,0.137739,519
4,14 South OR,113.963182,15.864849,0.148417,1025
5,05 South OR,58.098631,13.725245,0.143118,34
6,15 South OR,107.163620,13.217494,0.158491,1060
7,OR 04 BRAN,49.743370,12.078573,0.215052,353
8,01 MOC OR,34.312428,11.825520,0.112738,678
9,03 MSURG OR,41.301999,11.549113,0.156303,118


In [14]:
duckdb.sql("""
select 
    operating_room, 
    avg(total_elapsed) as avg_total_elapsed,
    stddev(total_elapsed) as stddev_total_elapsed,
    avg(wait) as avg_wait,
    stddev(wait) as stddev_wait, 
    avg(percent_of_wait) as avg_percent_of_wait,
    stddev(percent_of_wait) as stddev_percent_of_wait,
    count(*)
from df 
group by 1 
having count(*) > 30
order by 5 desc""").df()

,operating_room,avg_total_elapsed,stddev_total_elapsed,avg_wait,stddev_wait,avg_percent_of_wait,stddev_percent_of_wait,count_star()
0,18 South OR,123.644737,82.994410,45.039474,20.900042,0.410982,0.126411,76
1,09 South OR,146.322581,111.326363,44.935484,19.106605,0.387351,0.154580,31
2,11 South OR,176.262032,86.628484,53.093583,17.286245,0.353090,0.147200,374
3,03 MOC OR,130.036609,57.250589,52.156069,16.734721,0.446875,0.137739,519
4,14 South OR,189.040000,113.963182,50.930732,15.864849,0.336847,0.148417,1025
5,05 South OR,108.941176,58.098631,38.735294,13.725245,0.404933,0.143118,34
6,15 South OR,182.267925,107.163620,49.463208,13.217494,0.340401,0.158491,1060
7,OR 04 BRAN,121.169972,49.743370,45.988669,12.078573,0.418227,0.215052,353
8,01 MOC OR,105.473451,34.312428,42.662242,11.825520,0.429032,0.112738,678
9,03 MSURG OR,85.644068,41.301999,37.050847,11.549113,0.486947,0.156303,118


In [15]:
duckdb.sql("""
select 
    *
from df 
order by wait desc""").df()

,OR Case Number,Admit Dt/Tm,Discharge Dt/Tm,Case Start Dt/Tm,Patient In Room Dt/Tm,Surgery Start Dt/Tm,Surgery Stop Dt/Tm,Pt Out of Room Dt/Tm,Operating Room (Surg),admit_dt,discharge_dt,operating_room,total_elapsed,surgery_start_wait,surgery_end_wait,surgery_active_time,wait,percent_of_wait
0,SOUTH-2023-4036,2023-02-17 09:46:00,2023-03-01 14:00:00,2023-02-18 09:15:00,2023-02-18 07:27:00,2023-02-18 09:15:00,2023-02-18 14:09:00,2023-02-18 15:09:00,14 South OR,2023-02-17 09:46:00,2023-03-01 14:00:00,14 South OR,462.0,108.0,60.0,294.0,168.0,0.363636
1,SOUTH-2023-11947,2023-05-20 15:18:00,2023-05-31 17:50:00,2023-05-26 08:32:00,2023-05-26 07:15:00,2023-05-26 08:32:00,2023-05-26 11:37:00,2023-05-26 12:41:00,11 South OR,2023-05-20 15:18:00,2023-05-31 17:50:00,11 South OR,326.0,77.0,64.0,185.0,141.0,0.432515
2,SOUTH-2023-11947,2023-05-20 15:18:00,2023-05-31 17:50:00,2023-05-26 08:32:00,2023-05-26 07:15:00,2023-05-26 08:32:00,2023-05-26 11:37:00,2023-05-26 12:41:00,11 South OR,2023-05-20 15:18:00,2023-05-31 17:50:00,11 South OR,326.0,77.0,64.0,185.0,141.0,0.432515
3,SOUTH-2023-20943,2023-08-28 21:21:00,2023-09-15 15:25:00,2023-09-02 14:17:00,2023-09-02 13:10:00,2023-09-02 14:17:00,2023-09-02 18:49:00,2023-09-02 19:49:00,14 South OR,2023-08-28 21:21:00,2023-09-15 15:25:00,14 South OR,399.0,67.0,60.0,272.0,127.0,0.318296
4,SOUTH-2023-17858,2023-07-29 09:24:00,2023-08-03 11:30:00,2023-07-30 15:01:00,2023-07-30 13:23:00,2023-07-30 15:01:00,2023-07-30 17:03:00,2023-07-30 17:25:00,15 South OR,2023-07-29 09:24:00,2023-08-03 11:30:00,15 South OR,242.0,98.0,22.0,122.0,120.0,0.495868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8451,MOC-2023-3333,2023-06-30 05:04:00,2023-06-30 09:35:00,2023-06-30 07:14:00,2023-06-30 07:10:00,2023-06-30 07:14:00,2023-06-30 07:21:00,2023-06-30 07:24:00,04 MOC OR,2023-06-30 05:04:00,2023-06-30 09:35:00,04 MOC OR,14.0,4.0,3.0,7.0,7.0,0.500000
8452,SOUTH-2023-16731,2023-07-18 03:16:00,2023-07-19 17:45:00,2023-07-18 10:43:00,2023-07-18 10:38:00,2023-07-18 10:43:00,2023-07-18 10:58:00,2023-07-18 11:00:00,14 South OR,2023-07-18 03:16:00,2023-07-19 17:45:00,14 South OR,22.0,5.0,2.0,15.0,7.0,0.318182
8453,MOC-2023-2565,2023-06-08 05:11:00,2023-06-08 08:13:00,2023-06-08 07:29:00,2023-06-08 07:17:00,2023-06-08 07:29:00,2023-06-08 07:36:00,2023-06-08 07:29:00,06 MOC OR,2023-06-08 05:11:00,2023-06-08 08:13:00,06 MOC OR,12.0,12.0,-7.0,7.0,5.0,0.416667
8454,SOUTH-2023-12319,2023-05-26 11:47:00,2023-06-07 14:30:00,2023-05-26 13:28:00,2023-05-26 13:28:00,2023-05-26 13:28:00,2023-05-26 13:28:00,2023-05-26 13:28:00,12 South OR,2023-05-26 11:47:00,2023-06-07 14:30:00,12 South OR,0.0,0.0,0.0,0.0,0.0,NaN


In [29]:
duckdb.sql("""SELECT 
    operating_room,
    sum(case when wait >= 60 then 1 else 0 end) wait_time_over_one_hour,
    sum(case when wait >= 60 then 1 else 0 end)/count(*) pct_wait_time_over_one_hour,
    count(*) number_of_cases
    FROM df GROUP BY 1 HAVING COUNT(*) > 30 ORDER BY 3 DESC
    """).df()    ##  OUTLIERS

,operating_room,wait_time_over_one_hour,pct_wait_time_over_one_hour,number_of_cases
0,03 MOC OR,168.0,0.323699,519
1,11 South OR,116.0,0.310160,374
2,14 South OR,267.0,0.260488,1025
3,09 South OR,8.0,0.258065,31
4,15 South OR,189.0,0.178302,1060
5,18 South OR,8.0,0.105263,76
6,OR 04 BRAN,30.0,0.084986,353
7,OR 02 BRAN,50.0,0.083195,601
8,01 MOC OR,56.0,0.082596,678
9,04 MOC OR,43.0,0.070147,613


In [34]:
df['percent_of_wait'].corr(df['wait'])

-0.32951363944193646

In [35]:
df['percent_of_wait'].corr(df['total_elapsed'])

-0.7202395933399938

In [36]:
df['wait'].corr(df['surgery_active_time'])

0.5693238064976952

In [37]:
df.count()

OR Case Number           8456
Admit Dt/Tm              8456
Discharge Dt/Tm          8399
Case Start Dt/Tm         8456
Patient In Room Dt/Tm    8456
Surgery Start Dt/Tm      8456
Surgery Stop Dt/Tm       8456
Pt Out of Room Dt/Tm     8456
Operating Room (Surg)    8456
admit_dt                 8456
discharge_dt             8399
operating_room           8456
total_elapsed            8456
surgery_start_wait       8456
surgery_end_wait         8456
surgery_active_time      8456
wait                     8456
percent_of_wait          8455
dtype: int64